In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/track.csv')
df.shape[0], df.columns

(10125,
 Index(['id', 'start_date', 'avg_speed', 'end_date', 'length', 'max_speed',
        'norm_fuel_consumed', 'type', 'end_address', 'start_address', 'points',
        'tracker_id'],
       dtype='object'))

In [3]:
# calculate duration and average speed for each track 
df['duration'] = (pd.to_datetime(df.end_date) - pd.to_datetime(df.start_date)).dt.total_seconds() / 3600
df['avg_speed_calculated'] = df.length / df['duration']

In [4]:
# no tracks with duration > 24 hours
df['duration'].max()

5.948888888888889

In [5]:
# check if there are mistakes in the start/end dates
df[df.end_date <= df.start_date]

,id,start_date,avg_speed,end_date,length,max_speed,norm_fuel_consumed,type,end_address,start_address,points,tracker_id,duration,avg_speed_calculated


In [6]:
# check if track ids are unique and are not reused for different trackers tracks
df.loc[df.duplicated(subset='id', keep=False), ['id', 'tracker_id']].sort_values(by='id')

,id,tracker_id
5473,3525.0,3036049
4370,3525.0,3036047
4371,3526.0,3036047
5474,3526.0,3036049
4372,3527.0,3036047
...,...,...
1459,19918.0,877767
1460,19919.0,877767
7923,19919.0,3036068
1461,19920.0,877767


In [7]:
df['track_finished_date'] = pd.to_datetime(df['end_date']).dt.date

We want to get statistics on the distance traveled, duration (active and idle time), and average speed for trackers separately for each day.

In [8]:
# select tracks that start on one day and end on another
# we need to split each of them in 2 parts (one for each day)
start = df[pd.to_datetime(df.start_date).dt.day != pd.to_datetime(df.end_date).dt.day].copy()
end = df[pd.to_datetime(df.start_date).dt.day != pd.to_datetime(df.end_date).dt.day].copy()
start['end_date'] = start['start_date'].apply(lambda x: x[:11] + '23:59:59')
end['start_date'] = end['end_date'].apply(lambda x: x[:11] + '00:00:00')

There are only 2 trackers that have information on fuel consumed, still it would be interesting to have this information visualised.

In [9]:
df[df.norm_fuel_consumed.notna()].tracker_id.value_counts()

tracker_id
877766    1012
877767     450
Name: count, dtype: int64

In [10]:

start['duration'] = (pd.to_datetime(start.end_date) - pd.to_datetime(start.start_date)).dt.total_seconds() / 3600
end['duration'] = (pd.to_datetime(end.end_date) - pd.to_datetime(end.start_date)).dt.total_seconds() / 3600
start['length_split'] = round(start['duration'] * start['avg_speed_calculated'], 2)
end['length_split'] = round(end['duration'] * end['avg_speed_calculated'], 2)
start['norm_fuel_consumed'] = round(start['norm_fuel_consumed'] * start['length_split'] / start['length'], 2)
end['norm_fuel_consumed'] = round(end['norm_fuel_consumed'] * end['length_split'] / end['length'], 2)
start = start.drop(columns={'length'}).rename(columns={'length_split' : 'length'})
end = end.drop(columns={'length'}).rename(columns={'length_split' : 'length'})

In [11]:
stats = (df.groupby(['track_finished_date', 'tracker_id'])
         .agg({'id': 'count'})
         .reset_index()
         .rename(columns={'id':'count_finished', 'track_finished_date': 'date'}))

In [12]:
df.columns

Index(['id', 'start_date', 'avg_speed', 'end_date', 'length', 'max_speed',
       'norm_fuel_consumed', 'type', 'end_address', 'start_address', 'points',
       'tracker_id', 'duration', 'avg_speed_calculated',
       'track_finished_date'],
      dtype='object')

In [13]:
start = start[['id', 'start_date', 'avg_speed', 'end_date', 'length', 'max_speed',
        'norm_fuel_consumed', 'type', 'end_address', 'start_address', 'points',
        'tracker_id', 'duration', 'avg_speed_calculated',
        'track_finished_date']]
end = end[['id', 'start_date', 'avg_speed', 'end_date', 'length', 'max_speed',
        'norm_fuel_consumed', 'type', 'end_address', 'start_address', 'points',
        'tracker_id', 'duration', 'avg_speed_calculated',
        'track_finished_date']]

In [14]:
df = pd.concat([df[pd.to_datetime(df.start_date).dt.day == pd.to_datetime(df.end_date).dt.day],
                start, 
                end]
              ).reset_index(drop=True)


In [15]:
df['track_active_date'] = pd.to_datetime(df['start_date']).dt.date
stats = stats.merge(df.groupby(['track_active_date', 'tracker_id'])
                    .agg({'id': 'count'})
                    .reset_index()
                    .rename(columns={'id':'count_active', 'track_active_date':'date'}), 
                            on=['date', 'tracker_id'], 
                            how='outer')

In [26]:
(df.groupby(['track_active_date', 'tracker_id'])
                    .agg({'duration': 'sum'})
                    .reset_index())

,track_active_date,tracker_id,duration
0,2024-06-26,877766,16.653611
1,2024-06-26,877767,19.047222
2,2024-06-26,877768,18.850278
3,2024-06-26,3036043,7.025833
4,2024-06-26,3036045,2.933333
...,...,...,...
339,2024-07-23,3036043,1.783056
340,2024-07-23,3036056,2.113611
341,2024-07-23,3036057,0.665556
342,2024-07-23,3036068,2.125556


In [27]:
stats = stats.merge(df.groupby(['track_active_date', 'tracker_id'])
                    .agg({'length': 'sum'})
                    .reset_index()
                    .rename(columns={'track_active_date':'date'}), 
                            on=['date', 'tracker_id'], 
                            how='outer')
stats = stats.merge(df.groupby(['track_active_date', 'tracker_id'])
                    .agg({'duration': 'sum'})
                    .reset_index()
                    .rename(columns={'track_active_date':'date'}), 
                            on=['date', 'tracker_id'], 
                            how='outer')
stats['idle_time'] = 24 - stats['duration']
stats['avg_speed'] = stats['length'] / stats['duration']

In [28]:
stats

,date,tracker_id,count_finished,count_active,length,duration,idle_time,avg_speed
0,2024-06-26,877766,48,49,724.92,16.653611,7.346389,43.529298
1,2024-06-26,877767,21,22,850.93,19.047222,4.952778,44.674756
2,2024-06-26,877768,72,73,805.80,18.850278,5.149722,42.747381
3,2024-06-26,3036043,55,55,302.33,7.025833,16.974167,43.031194
4,2024-06-26,3036045,1,1,300.82,2.933333,21.066667,102.552273
...,...,...,...,...,...,...,...,...
339,2024-07-23,3036043,14,14,73.94,1.783056,22.216944,41.468141
340,2024-07-23,3036056,13,13,92.33,2.113611,21.886389,43.683533
341,2024-07-23,3036057,1,1,64.96,0.665556,23.334444,97.602671
342,2024-07-23,3036068,13,13,92.23,2.125556,21.874444,43.391009


In [32]:
stats.loc[stats.tracker_id == 877766], ['date', 'length'].astype({'date':'str'})

SyntaxError: unmatched ']' (2109545264.py, line 1)

In [60]:
stats.columns, dfm.columns

(Index(['date', 'tracker_id', 'count_finished', 'count_active', 'length',
        'duration', 'idle_time', 'avg_speed'],
       dtype='object'),
 Index(['tracker_id', 'date', 'mileage'], dtype='object'))

In [80]:
dfm = pd.read_csv('data/mileage.csv')
dfm = (dfm.sort_values(by='date').reset_index(drop=True).astype({'date':'str'})
        .merge(stats[['date', 'tracker_id', 'length', 'count_finished', 'count_active']].sort_values(by='date').reset_index(drop=True).astype({'date':'str'}), 
                on=['tracker_id', 'date'], how='outer'))
dfm['length_diff'] = abs(dfm.mileage - dfm.length)
dfm[dfm['length_diff'] > 2].sort_values(by='length_diff', ascending=False).head(10)

,tracker_id,date,mileage,length,count_finished,count_active,length_diff
292,3036057,2024-07-18,1385.05,1144.22,4.0,4.0,240.83
276,3036057,2024-07-02,1330.81,1091.56,3.0,3.0,239.25
412,3036069,2024-07-18,1190.73,987.96,6.0,6.0,202.77
410,3036069,2024-07-16,1235.33,1069.83,6.0,6.0,165.50
402,3036069,2024-07-08,1075.85,923.74,4.0,4.0,152.11
286,3036057,2024-07-12,1143.88,993.86,3.0,3.0,150.02
406,3036069,2024-07-12,1040.31,903.60,4.0,4.0,136.71
170,3036045,2024-07-16,549.50,429.11,2.0,2.0,120.39
282,3036057,2024-07-08,1531.56,1428.01,4.0,4.0,103.55
416,3036069,2024-07-22,921.15,971.57,5.0,6.0,50.42


In [77]:
dfm[dfm['length_diff'] > 2].tracker_id.nunique()

14

In [79]:
dfm[(dfm['length_diff'] > 2) & (dfm.count_finished != dfm.count_active)].shape[0]/dfm[dfm['length_diff'] > 2].shape[0]

0.22826086956521738

In [81]:
with pd.option_context("display.max_rows", 1200, "display.max_columns", 1000, 'display.max_colwidth', 1000):
    display(df.loc[df.start_date.str.contains('2024-07-18') & (df.tracker_id == 3036057), 
            ['id', 'start_date', 'end_date', 'length', 'start_address', 'end_address']])

,id,start_date,end_date,length,start_address,end_address
6667,360.0,2024-07-18 05:15:26,2024-07-18 09:31:29,411.83,"Carretera Matehuala-San Luis Potosí, San Luis Potosí City, San Luis Potosí, Mexico, 78319","Avenida Constituyentes - General José Montesinos, Avenida Constituyentes, Mexico City, Miguel Hidalgo, Mexico, 11100"
6668,361.0,2024-07-18 09:31:34,2024-07-18 09:53:48,27.33,"Avenida Constituyentes - General José Montesinos, Avenida Constituyentes, Mexico City, Miguel Hidalgo, Mexico, 11100","Viaducto Bicentenario, Tlalnepantla, State of Mexico, Mexico, 54015"
6669,362.0,2024-07-18 11:06:02,2024-07-18 14:10:19,297.18,"Carretera Querétaro - San Luis Potosí, Punto Blanco Dos, Guanajuato, Mexico, 37914","Avenida Constituyentes - General José Montesinos, Avenida Constituyentes, Mexico City, Miguel Hidalgo, Mexico, 11100"
6670,363.0,2024-07-18 14:10:24,2024-07-18 18:21:12,407.88,"Avenida Constituyentes - General José Montesinos, Avenida Constituyentes, Mexico City, Miguel Hidalgo, Mexico, 11100","Calle José María Mercado, San Luis Potosí City, San Luis Potosí, Mexico, 78319"


In [82]:
with pd.option_context("display.max_rows", 1200, "display.max_columns", 1000, 'display.max_colwidth', 1000):
    display(df.loc[df.start_date.str.contains('2024-07-08') & (df.tracker_id == 3036069), 
            ['id', 'start_date', 'end_date', 'length', 'start_address', 'end_address']])

,id,start_date,end_date,length,start_address,end_address
8457,905.0,2024-07-08 07:41:26,2024-07-08 10:47:26,246.69,"A 3, Kitzingen, Bavaria, Germany, 97318","A 3, Dernbach, Puderbach, Landkreis Neuwied, Rhineland-Palatinate, Germany, 56307"
8458,906.0,2024-07-08 11:33:03,2024-07-08 14:34:30,235.62,"A 3, Bischbrunn, Verwaltungsgemeinschaft Marktheidenfeld, Bavaria, Germany, 97836","A 3, Rösrath, North Rhine-Westphalia, Germany, 51503"
8459,907.0,2024-07-08 14:39:00,2024-07-08 14:49:41,1.70,"A 3, Rösrath, North Rhine-Westphalia, Germany, 51503","A 3, Rösrath, North Rhine-Westphalia, Germany, 51503"
8460,908.0,2024-07-08 14:50:21,2024-07-08 19:43:52,439.73,"A 3, Rösrath, North Rhine-Westphalia, Germany, 51503","A 3, Kitzingen, Bavaria, Germany, 97318"


In [84]:
stats.to_csv('data/stats.csv', index=False)

In [19]:
# with pd.option_context("display.max_rows", 1200, "display.max_columns", 1000, 'display.max_colwidth', 1000):
#     display(df[['avg_speed', 'avg_speed_calculated']])

In [20]:
# df.groupby(['tracker_id'])[['avg_speed', 'length', 'norm_fuel_consumed', 'points']].sum()